In [1]:
import pandas as pd
import math
import random
import re
import json
import numpy as np
from matplotlib import pyplot as plt
import io
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import sklearn.utils
from sklearn.linear_model import LogisticRegression
from google.colab import files
from google.colab import drive
from numpy.linalg import norm
from sklearn.svm import SVC
import pickle
from sklearn.metrics import recall_score, accuracy_score, f1_score, precision_score
drive.mount('/drive')

df_questions = pd.read_pickle("/drive/MyDrive/rp_data/duplicate_questions_exp_prep4.pkl")
df_pairs = pd.read_pickle("/drive/MyDrive/rp_data/duplicate_pairs_exp_prep4.pkl")

Mounted at /drive


In [2]:
# Adding the hier
def use_hierarchy_mod():
    file_hier = open('/drive/MyDrive/rp_data/json/mod_no_weights_res_1_cluster_hierarchy_test.json')
    file_labels = open('/drive/MyDrive/rp_data/json/mod_labels.json')
    data_hier = json.load(file_hier)
    data_labels = json.load(file_labels)
    return data_hier, data_labels

def use_hierarchy_manual():
  file_hier = open('/drive/MyDrive/rp_data/json/manual_hier.json')
  file_labels = open('/drive/MyDrive/rp_data/json/manual_labels.json')
  data_hier = json.load(file_hier)
  data_labels = json.load(file_labels)
  return data_hier, data_labels

def use_hierarchy_stat():
  file_hier = open('/drive/MyDrive/rp_data/json/statistical_inference_no_weights_cluster_hierarchy_test.json')
  file_labels = open('/drive/MyDrive/rp_data/json/stat_labels.json')
  data_hier = json.load(file_hier)
  data_labels = json.load(file_labels)
  return data_hier, data_labels

def use_hierarchy_auto():
  file_hier = open('/drive/MyDrive/rp_data/json/automatic_hier.json')
  file_labels = open('/drive/MyDrive/rp_data/json/automatic_labels.json')
  data_hier = json.load(file_hier)
  data_labels = json.load(file_labels)
  return data_hier, data_labels

def dfs_start(dict_hier, dict_labels, x, y):
    score = 0
    if x in dict_hier['value']['content'] and y in dict_hier['value']['content']:
        score = float(dict_labels[str(dict_hier['value']['uniqueId'])]['weight'])
    for child in dict_hier['children']:
        score_child = dfs_start(child, dict_labels, x, y)
        score = max(score_child, score)
    return score
    
def compute_score(dict_hier, dict_labels, tags1, tags2):
    score = 0
    for x in tags1:
        for y in tags2:
            score = max(score, dfs_start(dict_hier, dict_labels, x, y))
    return score

In [3]:
#### THIS CELL IS FOR DOC2VEC embeddings
# def train_embeddings(data, column, dim):
#     tagged_data = [TaggedDocument(words=words, tags=[i]) for i, words in enumerate(data)]

#     max_epochs = 70
#     vec_size = dim
#     alpha = 0.025

#     model = Doc2Vec(vector_size=vec_size,
#                     alpha=alpha,
#                     min_alpha=0.00025,
#                     min_count=1,
#                     dm=1, negative=5)

#     model.build_vocab(tagged_data)

#     for epoch in range(max_epochs):
#         print('iteration {0}'.format(epoch))
#         model.train(tagged_data,
#                     total_examples=model.corpus_count,
#                     epochs=1)
#         # decrease the learning rate
#         model.alpha -= 0.0002
#         # fix the learning rate, no decay
#         model.min_alpha = model.alpha

#     model_name = "d2v_v4.model_{}".format(column)
    
#     model_name = "/drive/MyDrive/rp_data/models/" + model_name
#     print(model_name)
#     model.save(model_name)
#     print("Model Saved")
#     return model, tagged_data

# def train_embeddings(id, data, column, dim, epochs):
#   tagged_data = [TaggedDocument(words=words, tags=[i]) for i, words in enumerate(data)]
#   model = Doc2Vec(dm = 0, min_count=3, vector_size = dim,  alpha = 0.025, min_alpha=0.00025, epochs = epochs)
#   model.build_vocab(tagged_data)
#   model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)
#   model_name = "d2v.model_{}_v{}".format(column, id)
#   model_name = "/drive/MyDrive/rp_data/models/" + model_name
#   print(model_name)
#   model.save(model_name)
#   print("Model Saved")
#   return model, tagged_data

  
def train_embeddings(id, data, column, dim, epochs):
  tagged_data = [TaggedDocument(words=words, tags=[i]) for i, words in enumerate(data)]
  model = Doc2Vec(dm = 1, dbow_words=1, min_count=3, vector_size = dim,  alpha = 0.025, min_alpha=0.00025, epochs = epochs)
  model.build_vocab(tagged_data)
  model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)
  model_name = "d2v.model_{}_v{}".format(column, id)
  model_name = "/drive/MyDrive/rp_data/models/" + model_name
  print(model_name)
  model.save(model_name)
  print("Model Saved")
  return model, tagged_data

# def train_embeddings(id, data, column, dim, epochs):
#   tagged_data = [TaggedDocument(words=words, tags=[i]) for i, words in enumerate(data)]
#   model = Doc2Vec(dm = 1, dbow_words=1, min_count=3, vector_size = dim,  alpha = 0.025, min_alpha=0.00025, epochs = epochs)
#   model_name = "d2v.model_{}_v{}".format(column, id)
#   model_name = "/drive/MyDrive/rp_data/models/" + model_name
#   print(model_name)
#   model.save(model_name)
#   print("Model Saved")
#   return model, tagged_data

def create_train_set(df_questions, set_questions_ids, column):
    list_data = []
    for i in df_questions.index:
        if df_questions['Id'][i] in set_questions_ids:
            list_data.append(df_questions[column][i])
    return list_data



def train_model(set_question_ids, dim_title, dim_body):
    # df_questions = pd.read_pickle("./data_rp/SO_files/duplicate_questions_v1_workaround_prep.pkl")
    list_questions_body = create_train_set(df_questions, set_question_ids, 'Body')
    list_questions_title = create_train_set(df_questions, set_question_ids, 'Title')

    # print("I am here")
    # print(type(list_questions_body))
    # print(list_questions_body)

    model_body, body_tagged = train_embeddings(list_questions_body, 'body', dim_body)
    model_title, title_tagged = train_embeddings(list_questions_title, 'title', dim_title)

In [4]:
def merge_2_lists(l1, l2):
    l = np.concatenate([l1, l2])
    return l.tolist()

def create_set_questions_df(df_pairs):
    set_questions = set()
    for x in df_pairs['PostId']:
        # print(x, "printam x")
        if x not in set_questions:
            set_questions.add(x)
    for x in df_pairs['RelatedPostId']:
        if x not in set_questions:
            set_questions.add(x)
    return set_questions

def create_merge_data(list_pairs, list_pairs_non, list_labels_pos, list_labels_neg):
    list_data = list_pairs + list_pairs_non
    list_labels = list_labels_pos + list_labels_neg

    list_data, list_labels = sklearn.utils.shuffle(list_data, list_labels, random_state=73)
    # list_data, list_labels = sklearn.utils.shuffle(list_data, list_labels)
    return list_data, list_labels


def create_list_pairs_and_labels(df_pairs, label):
    list_pairs = []
    for i in df_pairs.index:
        list_pairs.append((df_pairs['PostId'][i], df_pairs['RelatedPostId'][i]))
    list_labels = [label for x in range(len(list_pairs))]
    return list_pairs, list_labels


def create_df_question_based_on_pairs(df_pairs, df_questions):
    set_questions = create_set_questions_df(df_pairs)
    return df_questions.loc[df_questions['Id'].isin(set_questions)]

def create_dataset_process(df_pairs_positive, df_pairs_negative):
    list_pos_pairs, list_labels_pos = create_list_pairs_and_labels(df_pairs_positive, 1)
    list_neg_pairs, list_labels_neg = create_list_pairs_and_labels(df_pairs_negative, 0)

    return create_merge_data(list_pos_pairs, list_neg_pairs, list_labels_pos, list_labels_neg)

def get_textual_info(df_questions, q_id, column):
    for i in df_questions.index:
        if df_questions['Id'][i] == q_id:
            return df_questions[column][i]

def generate_pair_embeddings(q1, q2, model_body, model_title):
    vec_body_q1 = model_body.infer_vector(get_textual_info(df_questions, q1, 'Body'))
    vec_body_q2 = model_body.infer_vector(get_textual_info(df_questions, q2, 'Body'))
   

    vec_title_q1 = model_title.infer_vector(get_textual_info(df_questions, q1, 'Title'))
    vec_title_q2 = model_title.infer_vector(get_textual_info(df_questions, q2, 'Title'))
    #For T1
    # vec_body = vec_body_q1 + vec_body_q2
    # vec_title = vec_title_q1 + vec_title_q2
    # return vec_title, vec_body

    #for T2
    vec_q1 = np.concatenate([vec_title_q1, vec_body_q1])
    vec_q2 = np.concatenate([vec_title_q2, vec_body_q2])
    return vec_q1, vec_q2

def create_dataset_lr(list_pairs, model_body, model_title):
    list_input = []
    for x, y in list_pairs:
        if x > y: x, y = y, x
        # For t1
        # te, be = generate_pair_embeddings(x, y, model_body, model_title)
        # print("Debug here")
        # print(merge_2_lists(te, be))

        ##For T2:
        te, be = generate_pair_embeddings(x, y, model_body, model_title)
        list_input.append(merge_2_lists(te, be))
    return list_input

def make_negative_pairs(df_pairs, df_questions):
    set_pairs = set()
    for i in df_pairs.index:
        x = df_pairs['PostId'][i]
        y = df_pairs['RelatedPostId'][i]
        if x < y:
            set_pairs.add((x, y))
        else:
            set_pairs.add((y, x))

    list_index_questions = df_questions.index.tolist()
    list_non_duplicate_pairs = []
    list_non_duplicate_pairs_x = []
    list_non_duplicate_pairs_y = []

    # print("I am here")
    # print("type of the ", type(df_questions.index.tolist()))
    # print(df_questions.index.tolist())

    while len(list_non_duplicate_pairs) < len(df_pairs.index.tolist()):
        pair_random = random.sample(list_index_questions, 2)
        q1_id = df_questions['Id'][pair_random[0]]
        q2_id = df_questions['Id'][pair_random[1]]
        if q1_id > q2_id:
            q1_id, q2_id = q2_id, q1_id

        if (q1_id, q2_id) not in set_pairs:
            list_non_duplicate_pairs.append((q1_id, q2_id))

    df_pairs_non = pd.DataFrame(list_non_duplicate_pairs, columns=['PostId', 'RelatedPostId'])
    return df_pairs_non
  
def create_df_question_based_on_set(set_questions, df_questions):
    return df_questions.loc[df_questions['Id'].isin(set_questions)]

def create_positive_pair_set(df_pairs):
    set_pairs = set()
    for i in df_pairs.index:
        x = df_pairs['PostId'][i]
        y = df_pairs['RelatedPostId'][i]
        if x > y:
            x, y, = y, x
        set_pairs.add((x, y))
    return set_pairs

def filter_based_on_hier_score(hier, labels, q_1, q_2, baseline_score, df_questions):
    tags1 = df_questions.loc[df_questions['Id'] == q_1]['Tags'].to_numpy()[0]
    tags2 = df_questions.loc[df_questions['Id'] == q_2]['Tags'].to_numpy()[0]
    print("Printing the tags")
    print(tags1)
    score = compute_score(hier, labels, tags1, tags2)
    return score >= baseline_score

def filter_based_on_tags_score(q_1, q_2, baseline_score, df_questions):
    tags1 = df_questions.loc[df_questions['Id'] == q_1]['Tags'].to_numpy()[0]
    tags2 = df_questions.loc[df_questions['Id'] == q_2]['Tags'].to_numpy()[0]
    score = set(tags1).intersection(set(tags2))
    return score >= baseline_score

def filter_based_on_tags_score_v2(set_q1_tags, q_2, baseline_score, df_questions):
    # tags2 = df_questions.loc[df_questions['Id'] == q_2]['Tags'].to_numpy()[0]
    tags2 = df_questions.loc[df_questions.index == q_2]['Tags'].to_numpy()[0]
    set2 = set(tags2)
    score = len(set_q1_tags.intersection(set2))
    return score >= baseline_score


def make_recommandations(ml_model, q_id, set_pairs, baseline_score, df_questions, df_questions_learning,  hier, labels, df_questions_body, df_questions_title):
    #here the hierarchy can be used for filtering
    # list_pairs, list_labels = create_all_pairs_for_rec_hier(df_questions, set_pairs, q_id, baseline_score, hier, labels)
    #here the tags can be used for filtering
    # tags_q_id = df_questions.loc[df_questions['Id'] == q_id]['Tags'].to_numpy()[0]
    tags_q_id = df_questions.loc[df_questions.index == q_id]['Tags'].to_numpy()[0]
    list_pairs, list_labels = create_all_pairs_for_rec_tags(df_questions_learning, set_pairs, q_id, tags_q_id, baseline_score)
    data_learning = create_dataset_pred_v2(list_pairs, df_questions_body, df_questions_title, q_id)
    # print(data_learning)
    predictions = ml_model.predict_proba(data_learning)[:, 1]
    # print("predictions", predictions)
    # print("I am here")
    output_data = sorted(zip(predictions, list_pairs, list_labels), reverse=True)
    # output_data.sort()
    return output_data

def labels_k_rec(ml_model, k, testing_questions, baseline_score, set_pairs, df_quesions, df_quesions_learning, hier, labels, df_questions_body, df_questions_title):
    list_pred = []
    list_true = []
    list_testing_data = []
    cnt =0
    for x in testing_questions:
        print(cnt)
        cnt = cnt + 1
        output_data = make_recommandations(ml_model, x, set_pairs, baseline_score, df_quesions, df_questions_learning, hier, labels, df_questions_body, df_questions_title)[:k]
        list_testing_data = list_testing_data + output_data
    return list_testing_data

def apply_confidence_threshold(confidence_threshold, testing_data):
    list_pred = []
    list_true = []
    for pred, p, l in testing_data:
            list_true.append(l)
            if pred >= confidence_threshold:
                list_pred.append(1)
            else:
                list_pred.append(0)
    return list_true, list_pred

def create_all_pairs_for_rec_hier(df_questions, set_positive_pairs, q_id, baseline_score, hier, labels):
    list_pairs = []
    list_labels = []
    tags_q_id = df_questions.loc[df_questions['Id'] == q_id]['Tags'].to_numpy()[0]
    for i in df_questions.index:
        q1_id = df_questions['Id'][i]
        if q_id == q1_id: continue
        q2_id = q_id
        if not filter_based_on_hier_score(hier, labels, q1_id, q2_id, baseline_score, df_questions): continue

        if q2_id < q1_id:
            q2_id, q1_id = q1_id, q2_id

        list_pairs.append((q1_id, q2_id))
        # print("I am here", (q1_id, q_id))
        if (q1_id, q2_id) in set_positive_pairs:
            list_labels.append(1)
        else:
            list_labels.append(0)

    return list_pairs, list_labels


def create_all_pairs_for_rec_tags(df_questions, set_positive_pairs, q_id, tags_q_id, baseline_score):
    list_pairs = []
    list_labels = []
    set_tags_q_id = set(tags_q_id)
    for i in df_questions.index:
        q1_id = i
        q2_id = q_id
        if not filter_based_on_tags_score_v2(set_tags_q_id, q1_id, baseline_score, df_questions): continue

        if q2_id < q1_id:
            q2_id, q1_id = q1_id, q2_id

        list_pairs.append((q1_id, q2_id))
        # print("I am here", (q1_id, q_id))
        if (q1_id, q2_id) in set_positive_pairs:
            list_labels.append(1)
        else:
            list_labels.append(0)

    return list_pairs, list_labels

def create_embedings_dataframe(df_questions, model, column):
  list_ids = []
  list_em = []
  for i in df_questions.index:
    x = df_questions['Id'][i]
    text = df_questions[column][i]
    vec = model.infer_vector(text)
    list_em.append(vec)
    list_ids.append(x)

  new_column = column + " Em"
  data = {'Id': list_ids, new_column: list_em}
  df = pd.DataFrame(data)
  return df

def create_post_embeddings_dataframe(df_questions, model_body, model_title):
  list_ids = []
  list_em = []
  for i in df_questions.index:
    x = df_questions['Id'][i]
    title_text = df_questions['Title'][i]
    body_text = df_questions['Body'][i]
    title = model_title.infer_vector(title_text)
    body = model_body.infer_vector(body_text)
    list_em.append(np.concatenate((title, body), axis = None))
    list_ids.append(x)
    # print(list_em)
    # break
  new_column = 'Embeddings'
  data = {'Id': list_ids, new_column: list_em}
  df = pd.DataFrame(data)
  return df

def create_dataset_lr_v2(list_pairs, df_questions_body, df_questions_title):
    list_input = []
    for x, y in list_pairs:
        if x > y: x, y = y, x
        # For t1
        # te, be = generate_pair_embeddings(x, y, model_body, model_title)
        # print("Debug here")
        # print(merge_2_lists(te, be))

        ##For T2:
        t1= df_questions_title[df_questions_title.index == x]['Title Em'].to_numpy()[0]
        t2= df_questions_title[df_questions_title.index == y]['Title Em'].to_numpy()[0]
        b1= df_questions_body[df_questions_body.index == x]['Body Em'].to_numpy()[0]
        b2= df_questions_body[df_questions_body.index == y]['Body Em'].to_numpy()[0]
      
        l = np.concatenate((t1,b1,t2,b2), axis=None)

        list_input.append(l)

    return list_input

def create_dataset_pred_v2(list_pairs, df_questions_body, df_questions_title, q_id):
    # t= df_questions_title[df_questions_title['Id'] == q_id]['Title Em'].to_numpy()[0]
    # b= df_questions_body[df_questions_body['Id'] == q_id]['Body Em'].to_numpy()[0]
    # list_input = []
    # for x, y in list_pairs:
    #     t1 = None
    #     b1 = None
    #     b2 = None
    #     t2 = None
    #     if x == q_id:
    #       t1 = t
    #       t2 = df_questions_title[df_questions_title['Id'] == y]['Title Em'].to_numpy()[0]
    #       b1 = b
    #       b2= df_questions_body[df_questions_body['Id'] == y]['Body Em'].to_numpy()[0]
    #     else: 
    #       t2 = t
    #       t1 = df_questions_title[df_questions_title['Id'] == y]['Title Em'].to_numpy()[0]
    #       b2 = b
    #       b1= df_questions_body[df_questions_body['Id'] == y]['Body Em'].to_numpy()[0]
    #     if x > y: 
    #       x, y = y, x
    #       t1, t2 = t2, t1
    #       b1, b2 = b2, b1

    #     l = np.concatenate((t1,b1,t2,b2), axis=None)
    #     list_input.append(l)

    # return list_input
    t= df_questions_title[df_questions_title.index == q_id]['Title Em'].to_numpy()[0]
    b= df_questions_body[df_questions_body.index == q_id]['Body Em'].to_numpy()[0]
    list_input = []
    for x, y in list_pairs:
        t1 = None
        b1 = None
        b2 = None
        t2 = None
        if x == q_id:
          t1 = t
          t2 = df_questions_title[df_questions_title.index == y]['Title Em'].to_numpy()[0]
          b1 = b
          b2= df_questions_body[df_questions_body.index == y]['Body Em'].to_numpy()[0]
        else: 
          t2 = t
          t1 = df_questions_title[df_questions_title.index == y]['Title Em'].to_numpy()[0]
          b2 = b
          b1= df_questions_body[df_questions_body.index == y]['Body Em'].to_numpy()[0]
        if x > y: 
          x, y = y, x
          t1, t2 = t2, t1
          b1, b2 = b2, b1

        l = np.concatenate((t1,b1,t2,b2), axis=None)
        list_input.append(l)

    return list_input

def sim_score_em_pair(q1, q2, df_questions_em):
  vec1 = df_questions_em[df_questions_em.index == q1]['Embeddings'].to_numpy()[0]
  vec2 = df_questions_em[df_questions_em.index == q2]['Embeddings'].to_numpy()[0]
  # print(vec1)
  cos_sim = np.dot(vec1, vec2)/(norm(vec1) * norm(vec2))
  return cos_sim

def create_output_data_sim(list_pairs, list_labels, df_questions_em):
  list_scores = []
 
  for x, y in list_pairs:
    list_scores.append(sim_score_em_pair(x, y, df_questions_em))
  # output_data = zip(list_scores, list_pairs, list_labels)
  output_data = sorted(zip(list_scores, list_pairs, list_labels), reverse=True)
  return output_data

def hier_score_calc(hier, labels, q_1, q_2, df_questions):
    tags1 = df_questions.loc[df_questions['Id'] == q_1]['Tags'].to_numpy()[0]
    tags2 = df_questions.loc[df_questions['Id'] == q_2]['Tags'].to_numpy()[0]
    return compute_score(hier, labels, tags1, tags2)
    # print("Printing the tags")
    # print(tags1)
    # score = compute_score(hier, labels, tags1, tags2)
    # return score >= baseline_score

def tags_score_calc(q_1, q_2, df_questions):
    tags1 = df_questions.loc[df_questions['Id'] == q_1]['Tags'].to_numpy()[0]
    tags2 = df_questions.loc[df_questions['Id'] == q_2]['Tags'].to_numpy()[0]
    return len(set(tags1).intersection(set(tags2)))/len(set(tags1).union(set(tags2)))
    # score = set(tags1).intersection(set(tags2))
    # return score >= baseline_score
def list_hier_score_calc(hier, labels, list_pairs, df_questions):
  list_scores = []
  for x, y in list_pairs:
    list_scores.append(hier_score_calc(hier, labels, x, y, df_questions))
  return  np.array(list_scores)

def list_tags_score_calc(list_pairs, df_questions):
  list_scores = []
  for x, y in list_pairs:
    list_scores.append(tags_score_calc(x, y, df_questions))
  return  np.array(list_scores)
  

In [5]:
set_positive_pairs = create_positive_pair_set(df_pairs)
# print("orint apirs", df_pairs)
df_pairs_training = df_pairs.head(8000)
df_pairs_others = df_pairs.tail(2000)
df_pairs_validation = df_pairs_others.head(1000)
df_pairs_testing = df_pairs_others.tail(1000)
questions_set_training = create_set_questions_df(df_pairs_training)
questions_set_validating = create_set_questions_df(df_pairs_validation)
questions_set_testing = create_set_questions_df(df_pairs_testing)

questions_set_learning = questions_set_training.union(questions_set_validating)
questions_set_testing_use = questions_set_testing.difference(questions_set_training.union(questions_set_validating))

In [6]:
print(len(questions_set_validating), "print the dimension of the validation set")
print(len(questions_set_validating.difference(questions_set_training)), "print the dimenstion of the questions from vlidation but not in training")

print(len(questions_set_testing.union(questions_set_validating.union(questions_set_training))), "print the number of questions which will be used for the evaluating")
print(len(questions_set_training), "print the dimension of the training set")

df_questions_training = create_df_question_based_on_pairs(df_pairs_training, df_questions)
df_questions_validation = create_df_question_based_on_pairs(df_pairs_validation, df_questions)

df_questions_learning = create_df_question_based_on_set(questions_set_learning, df_questions)
# df_questions_learning.set_index('Id', inplace =True)

df_non_pairs_training = make_negative_pairs(df_pairs_training, df_questions_training)
df_non_pairs_validation = make_negative_pairs(df_pairs_validation, df_questions_validation)


# df_questions_testing = create_df_question_based_on_set(questions_set_testing_use, df_questions)
#Create the pairs for testing
df_questions_testing = create_df_question_based_on_pairs(df_pairs_testing, df_questions)
df_non_pairs_testing = make_negative_pairs(df_pairs_testing, df_questions_testing)
list_testing_pairs, list_testing_labels = create_dataset_process(df_pairs_testing, df_non_pairs_testing)
list_positive_testing_pairs, list_positive_testing_labels = create_dataset_process(df_pairs_testing, pd.DataFrame())
list_train_pairs, list_train_labels = create_dataset_process(df_pairs_training, df_non_pairs_training)
list_validation_pairs, list_validation_labels = create_dataset_process(df_pairs_validation, df_non_pairs_validation)




# def evaluate_embedding_models(df_questions_em_body, df_questions_em_title, df_questions):
  
#   #Create the datasets
#   # df_questions_em_body = create_embedings_dataframe(df_questions, model_body, 'Body')
#   # df_questions_em_body.to_pickle("/drive/MyDrive/rp_data/pickle/duplicate_questions_exp_em_body_doc2vec_v4.pkl")
#   # df_questions_em_title = create_embedings_dataframe(df_questions, model_title, 'Title')
#   # df_questions_em_title.to_pickle("/drive/MyDrive/rp_data/pickle/duplicate_questions_exp_em_title_doc2vec_v4.pkl")
#   # df_questions_em = create_post_embeddings_dataframe(df_questions, model_body, model_title)
#   # df_questions_em_body.set_index('Id', inplace = True)
#   # df_questions_em_title.set_index('Id', inplace = True)
#   # df_questions.set_index('Id', inplace =True)
#   # df_questions_em.set_index('Id', inplace =True)
#   # print("Start making the TRAINING dataset for LR")
#   train_data_lr = create_dataset_lr_v2(list_train_pairs,df_questions_em_body, df_questions_em_title)
#   # print("Start making the VALIDATION dataset for LR")
#   validation_data_lr = create_dataset_lr_v2(list_validation_pairs,df_questions_em_body, df_questions_em_title)
#   # print("Finish making the dataset for LR")

#   lr = LogisticRegression(penalty = 'l1', max_iter = 200, solver = 'saga')
#   lr.fit(train_data_lr, list_train_labels)
#   list_validation_labels_pred = lr.predict(validation_data_lr)

#   accuracy = accuracy_score(list_validation_labels, list_validation_labels_pred)
#   recall = recall_score(list_validation_labels, list_validation_labels_pred)
#   f1 = f1_score(list_validation_labels, list_validation_labels_pred)
#   precision = precision_score(list_validation_labels, list_validation_labels_pred)
#   # print(recall) 
#   # print(accuracy)
#   # print(precision)
#   # print(f1)

#   return recall, accuracy, precision, f1

def evaluate_embedding_models(df_questions_em_body, df_questions_em_title):
    # Create the datasets
    # df_questions_em_body = create_embedings_dataframe(df_questions, model_body, 'Body')
    # df_questions_em_body.to_pickle("/drive/MyDrive/rp_data/pickle/duplicate_questions_exp_em_body_doc2vec_v4.pkl")
    # df_questions_em_title = create_embedings_dataframe(df_questions, model_title, 'Title')
    # df_questions_em_title.to_pickle("/drive/MyDrive/rp_data/pickle/duplicate_questions_exp_em_title_doc2vec_v4.pkl")
    # df_questions_em = create_post_embeddings_dataframe(df_questions, model_body, model_title)
    # df_questions_em_body.set_index('Id', inplace = True)
    # df_questions_em_title.set_index('Id', inplace = True)
    # df_questions.set_index('Id', inplace =True)
    # df_questions_em.set_index('Id', inplace =True)
    # print("Start making the TRAINING dataset for LR")
    train_data_lr = create_dataset_lr_v2(list_train_pairs, df_questions_em_body, df_questions_em_title)
    # print("Start making the VALIDATION dataset for LR")
    validation_data_lr = create_dataset_lr_v2(list_validation_pairs, df_questions_em_body, df_questions_em_title)
    # print("Finish making the dataset for LR")
    #Make all the data numpy
    train_data_np = np.array(train_data_lr)
    train_labels_np = np.array(list_train_labels)
    validation_data_np = np.array(validation_data_lr)
    validation_labels_np = np.array(list_validation_labels)


    lr = LogisticRegression(random_state = 42)
    lr.fit(train_data_np, train_labels_np)
    prediction = lr.predict(validation_data_np)
    f1 = f1_score(prediction,validation_labels_np, average="weighted")
    accuracy = accuracy_score(prediction, validation_labels_np)
    recall = recall_score(prediction, validation_labels_np)
    precision = precision_score(prediction, validation_labels_np)



    ######THE OLD VERSIONS - SLOW
    # lr = LogisticRegression(penalty='l1', max_iter=200, solver='saga')
    # lr.fit(train_data_lr, list_train_labels)
    # list_validation_labels_pred = lr.predict(validation_data_lr)

    # lr = LogisticRegression()
    # lr.fit(train_data_lr, list_train_labels)
    # list_validation_labels_pred = lr.predict(validation_data_lr)

    # accuracy = accuracy_score(list_validation_labels, list_validation_labels_pred)
    # recall = recall_score(list_validation_labels, list_validation_labels_pred)
    # f1 = f1_score(list_validation_labels, list_validation_labels_pred)
    # precision = precision_score(list_validation_labels, list_validation_labels_pred)
    # print(recall)
    # print(accuracy)
    # print(precision)
    # print(f1)

    return recall, accuracy, precision, f1



# def create_list_dfs_em_body(list_dfs_em_body):
#   # list_dfs_em_body = []
#   # print("start body")
#   for i in range(88):
#       # if i == 0: print("satr body 1")
#       model_body =  Doc2Vec.load('/drive/MyDrive/rp_data/models/d2v.model_body_v{}'.format(i))
#       df_questions_em_body = create_embedings_dataframe(df_questions, model_body, 'Body')
#       df_questions_em_body.set_index('Id', inplace = True)
#       list_dfs_em_body.append(df_questions_em_body)
#   # return list_dfs_em_body

# def create_list_dfs_em_title(list_dfs_em_title):
#   # list_dfs_em_title = []
#   # print("start body")
#   for j in range(56):
#     # if j == 0: print("satr body 1")
#     model_title =  Doc2Vec.load('/drive/MyDrive/rp_data/models/d2v.model_title_v{}'.format(j))
#     df_questions_em_title = create_embedings_dataframe(df_questions, model_title, 'Title')
#     df_questions_em_title.set_index('Id', inplace = True)
#     list_dfs_em_title.append(df_questions_em_title)
#   # return list_dfs_em_title

def create_list_dfs_em_body(list_dfs_em_body, start, stop):
    # for i in range(start, start + 2):
    #     # if i == 0: print("satr body 1")
    #     # model_body = Doc2Vec.load('./data_rp/models_google/d2v.model_title_v{}'.format(i))
    #     model_body = Doc2Vec.load('./data_rp/models_local/d2v.model_title_v{}'.format(i))
    #     df_questions_em_body = create_embedings_dataframe(df_questions, model_body, 'Body')
    #     df_questions_em_body.set_index('Id', inplace=True)
    #     list_dfs_em_body.append((i, df_questions_em_body))
    #     print(i, "Body")
    # # short_version

    # list_dfs_em_body = []
    print("Start body {}".format(start))
    for i in range(start, stop):
        # if i == 0: print("satr body 1")
        model_body = Doc2Vec.load('/drive/MyDrive/rp_data/models/d2v.model_body_v{}'.format(i))
        # model_body = Doc2Vec.load('./data_rp/models_local/d2v.model_body_v{}'.format(i))
        df_questions_em_body = create_embedings_dataframe(df_questions, model_body, 'Body')
        df_questions_em_body.set_index('Id', inplace=True)
        print(i, "Body")
        list_dfs_em_body.append((i, df_questions_em_body))
    print("Finish body {}".format(start))

def create_list_dfs_em_title(list_dfs_em_title, start, stop):
    # for j in range(start, start + 2):
    #     # if j == 0: print("satr body 1")
    #     # model_title = Doc2Vec.load('./data_rp/models_google/d2v.model_title_v{}'.format(j))
    #     model_title = Doc2Vec.load('./data_rp/models_local/d2v.model_title_v{}'.format(j))
    #     df_questions_em_title = create_embedings_dataframe(df_questions, model_title, 'Title')
    #     df_questions_em_title.set_index('Id', inplace=True)
    #     print(j, "Title")
    #     list_dfs_em_title.append((j, df_questions_em_title))
    # short_version
    #
    # list_dfs_em_title = []
    # # print("start body")
    print("Start title {}".format(start))
    for j in range(start, stop):
        # if j == 0: print("satr body 1")
        model_title = Doc2Vec.load('/drive/MyDrive/rp_data/models/d2v.model_title_v{}'.format(j))
        # model_title = Doc2Vec.load('./data_rp/models_local/d2v.model_title_v{}'.format(j))
        df_questions_em_title = create_embedings_dataframe(df_questions, model_title, 'Title')
        df_questions_em_title.set_index('Id', inplace=True)
        list_dfs_em_title.append((j, df_questions_em_title))
        print(j, "Title")
    print("Finish title {}".format(start))


def create_eval_dict_embeddings(df_questions):
  evaluations_em = []
  list_dfs_em_title1 = []
  list_dfs_em_title2 = []
  list_dfs_em_body1 = []
  list_dfs_em_body2 = []

  create_list_dfs_em_title(list_dfs_em_title1, 15, 21)
  create_list_dfs_em_title(list_dfs_em_title2, 43, 49)
  create_list_dfs_em_body(list_dfs_em_body1, 26, 33)
  create_list_dfs_em_body(list_dfs_em_body2, 70, 77)

  print("Title done")
  # list_dfs_em_title = list_dfs_em_title1 + list_dfs_em_title2 + list_dfs_em_title3 + list_dfs_em_title4
  list_dfs_em_title = list_dfs_em_title1 + list_dfs_em_title2

  print("Body done")
  # list_dfs_em_body = list_dfs_em_body1 + list_dfs_em_body2 + list_dfs_em_body3 + list_dfs_em_body4
  list_dfs_em_body = list_dfs_em_body1 + list_dfs_em_body2

  # #Search for all
  # for i in range(88):
  #     for j in range(56):
  # Optimized Search
  for i, df_body in list_dfs_em_body:
      for j, df_title in list_dfs_em_title:
          # for i in range(2):
          #     for j in range(2):
          # model_body =  Doc2Vec.load('/drive/MyDrive/rp_data/models/d2v.model_body_v{}'.format(i))
          # model_title =  Doc2Vec.load('/drive/MyDrive/rp_data/models/d2v.model_title_v{}'.format(j))

          recall, accuracy, precision, f1 = evaluate_embedding_models(df_body, df_title)
          evaluation = {
              'em_body_id': i,
              'em_title_id': j,
              'recall': recall,
              'accuracy': accuracy,
              'precision': precision,
              'f1': f1
          }
          evaluations_em.append(evaluation)
          print(len(evaluations_em))
  return evaluations_em
  



# evaluations_em = create_eval_dict_embeddings(df_questions)


    



# print("Start making the TRAINING dataset for LR")
# train_data_lr = create_dataset_lr(list_train_pairs, model_body, model_title)
# print("Start making the VALIDATION dataset for LR")
# validation_data_lr = create_dataset_lr(list_validation_pairs, model_body, model_title)
# print("Finish making the dataset for LR")

1914 print the dimension of the validation set
1722 print the dimenstion of the questions from vlidation but not in training
18075 print the number of questions which will be used for the evaluating
14627 print the dimension of the training set


In [ ]:

print("Starting the evaluation using ML")
# evaluations_em = create_evaluate_dict_scores_embeddings(list_dfs_em_body, list_dfs_em_title)
# with open("test_pickle.pkl", "wb") as fp:  # Pickling
with open("/drive/MyDrive/rp_data/pickle/evalautions_em_doc2vec_full.pkl", "wb") as fp:  # Pickling
    pickle.dump(evaluations_em, fp)
print(evaluations_em)

Starting the evaluation using ML
[{'em_body_id': 26, 'em_title_id': 15, 'recall': 0.5541706615532119, 'accuracy': 0.5565, 'precision': 0.578, 'f1': 0.5567051026837155}, {'em_body_id': 26, 'em_title_id': 16, 'recall': 0.547550432276657, 'accuracy': 0.5495, 'precision': 0.57, 'f1': 0.5496894022212835}, {'em_body_id': 26, 'em_title_id': 17, 'recall': 0.5371057513914657, 'accuracy': 0.54, 'precision': 0.579, 'f1': 0.5407007258039478}, {'em_body_id': 26, 'em_title_id': 18, 'recall': 0.5474178403755868, 'accuracy': 0.5505, 'precision': 0.583, 'f1': 0.550975286396256}, {'em_body_id': 26, 'em_title_id': 19, 'recall': 0.5464114832535886, 'accuracy': 0.5485, 'precision': 0.571, 'f1': 0.5487286876481219}, {'em_body_id': 26, 'em_title_id': 20, 'recall': 0.5396975425330813, 'accuracy': 0.542, 'precision': 0.571, 'f1': 0.5423855022073565}, {'em_body_id': 26, 'em_title_id': 43, 'recall': 0.5608308605341247, 'accuracy': 0.5615, 'precision': 0.567, 'f1': 0.5615132650262672}, {'em_body_id': 26, 'em_titl

In [ ]:
sorted_evaluations = sorted(evaluations_em, key=lambda value: value['recall'], reverse=False)
print(sorted_evaluations)

[{'em_body_id': 30, 'em_title_id': 17, 'recall': 0.5345622119815668, 'accuracy': 0.5375, 'precision': 0.58, 'f1': 0.5383369022797428}, {'em_body_id': 75, 'em_title_id': 47, 'recall': 0.5363805970149254, 'accuracy': 0.539, 'precision': 0.575, 'f1': 0.5395982313077748}, {'em_body_id': 26, 'em_title_id': 17, 'recall': 0.5371057513914657, 'accuracy': 0.54, 'precision': 0.579, 'f1': 0.5407007258039478}, {'em_body_id': 29, 'em_title_id': 15, 'recall': 0.5380228136882129, 'accuracy': 0.54, 'precision': 0.566, 'f1': 0.5403111703511574}, {'em_body_id': 32, 'em_title_id': 17, 'recall': 0.5381040892193308, 'accuracy': 0.541, 'precision': 0.579, 'f1': 0.5416637544614423}, {'em_body_id': 32, 'em_title_id': 20, 'recall': 0.5385329619312906, 'accuracy': 0.5415, 'precision': 0.58, 'f1': 0.5421806204746986}, {'em_body_id': 29, 'em_title_id': 45, 'recall': 0.5387634936211972, 'accuracy': 0.5395, 'precision': 0.549, 'f1': 0.5395415638761399}, {'em_body_id': 29, 'em_title_id': 44, 'recall': 0.539575289575

In [ ]:
with open("/drive/MyDrive/rp_data/pickle/evalautions_em_doc2vec_full.pkl", "rb") as fp:  # Pickling
      evaluations_em_2  = pickle.load(fp)
evaluations_em_2
sorted_evaluations_2 = sorted(evaluations_em_2, key=lambda value: value['recall'], reverse=True)
print(sorted_evaluations_2)

[{'em_body_id': 72, 'em_title_id': 15, 'recall': 0.5635673624288425, 'accuracy': 0.567, 'precision': 0.594, 'f1': 0.5673158872818285}, {'em_body_id': 72, 'em_title_id': 16, 'recall': 0.5633270321361059, 'accuracy': 0.567, 'precision': 0.596, 'f1': 0.5673644595104483}, {'em_body_id': 30, 'em_title_id': 48, 'recall': 0.5619596541786743, 'accuracy': 0.5645, 'precision': 0.585, 'f1': 0.5646830958210188}, {'em_body_id': 26, 'em_title_id': 48, 'recall': 0.5616438356164384, 'accuracy': 0.563, 'precision': 0.574, 'f1': 0.5630528833988913}, {'em_body_id': 70, 'em_title_id': 16, 'recall': 0.5616438356164384, 'accuracy': 0.563, 'precision': 0.574, 'f1': 0.5630528833988913}, {'em_body_id': 26, 'em_title_id': 43, 'recall': 0.5608308605341247, 'accuracy': 0.5615, 'precision': 0.567, 'f1': 0.5615132650262672}, {'em_body_id': 70, 'em_title_id': 44, 'recall': 0.5607385811467445, 'accuracy': 0.5625, 'precision': 0.577, 'f1': 0.5625920037187819}, {'em_body_id': 27, 'em_title_id': 48, 'recall': 0.56064947

After sorting the embeddings, let's evaluate the models

In [7]:
# Model to evaluate - rank 1 -> full 
# doc2vec_em_title = Doc2Vec.load('/drive/MyDrive/rp_data/models/d2v.model_title_v{}'.format(48))
# doc2vec_em_body = Doc2Vec.load('/drive/MyDrive/rp_data/models/d2v.model_body_v{}'.format(30))

# df_questions_em_body = create_embedings_dataframe(df_questions, doc2vec_em_body, 'Body')
df_questions_em_body = pd.read_pickle("/drive/MyDrive/rp_data/pickle/duplicate_questions_exp_em_body_doc2vec_full_rank3.pkl")
# df_questions_em_body.to_pickle("/drive/MyDrive/rp_data/pickle/duplicate_questions_exp_em_body_doc2vec_full_rank1.pkl")
# df_questions_em_body.to_pickle("/drive/MyDrive/rp_data/pickle/duplicate_questions_exp_em_body_doc2vec_rank1.pkl")
# df_questions_em_body= pd.read_pickle("/drive/MyDrive/rp_data/pickle/duplicate_questions_exp_em_body_doc2vec_rank1.pkl")
# df_questions_em_title= pd.read_pickle("/drive/MyDrive/rp_data/pickle/duplicate_questions_exp_em_title_doc2vec_rank1.pkl")
# df_questions_em_title = create_embedings_dataframe(df_questions, doc2vec_em_title, 'Title')
df_questions_em_title = pd.read_pickle("/drive/MyDrive/rp_data/pickle/duplicate_questions_exp_em_title_doc2vec_full_rank3.pkl")
# df_questions_em_title.to_pickle("/drive/MyDrive/rp_data/pickle/duplicate_questions_exp_em_title_doc2vec_full_rank1.pkl")
# df_questions_em_title.to_pickle("/drive/MyDrive/rp_data/pickle/duplicate_questions_exp_em_title_doc2vec_rank1.pkl")
# df_questions_em = create_post_embeddings_dataframe(df_questions, doc2vec_em_body, doc2vec_em_title)
df_questions_em_body.set_index('Id', inplace = True)
df_questions_em_title.set_index('Id', inplace = True)
# df_questions_em.set_index('Id', inplace =True)

train_data_lr = create_dataset_lr_v2(list_train_pairs,df_questions_em_body, df_questions_em_title)
#   # print("Start making the VALIDATION dataset for LR")
validation_data_lr = create_dataset_lr_v2(list_validation_pairs,df_questions_em_body, df_questions_em_title)
#   # print("Finish making the dataset for LR")
testing_data_lr = create_dataset_lr_v2(list_testing_pairs, df_questions_em_body, df_questions_em_title)


### Create the datasets for the evaluating
learning_data_lr = merge_2_lists(train_data_lr, validation_data_lr)
list_learning_pairs = merge_2_lists(list_train_pairs, list_validation_pairs)
list_learning_labels = merge_2_lists(list_train_labels, list_validation_labels)
testing_data_lr = create_dataset_lr_v2(list_testing_pairs, df_questions_em_body, df_questions_em_title)
#Make all the data numpy
train_data_np = np.array(train_data_lr)
train_labels_np = np.array(list_train_labels)
validation_data_np = np.array(validation_data_lr)
validation_labels_np = np.array(list_validation_labels)
learning_data_np = np.array(learning_data_lr)
learning_labels_np = np.array(list_learning_labels)
testing_data_np = np.array(testing_data_lr)
testing_labels_np = np.array(list_testing_labels)

In [8]:
######CELL FOR CREATING THE TAGS##############
tags_output_testing_np = list_tags_score_calc(list_testing_pairs, df_questions)
tags_output_train_np = list_tags_score_calc(list_learning_pairs, df_questions)

######CELL FOR CREATING THE HIERARCHY##############
data_hier_stat, data_labels_stat = use_hierarchy_stat()
hier_stat_output_testing_np = list_hier_score_calc(data_hier_stat, data_labels_stat, list_testing_pairs, df_questions)
hier_stat_output_train_np = list_hier_score_calc(data_hier_stat, data_labels_stat, list_learning_pairs, df_questions)

data_hier_manual, data_labels_manual = use_hierarchy_manual()
hier_manual_output_testing_np = list_hier_score_calc(data_hier_manual, data_labels_manual, list_testing_pairs, df_questions)
hier_manual_output_train_np = list_hier_score_calc(data_hier_manual, data_labels_manual, list_learning_pairs, df_questions)

data_hier_mod, data_labels_mod = use_hierarchy_mod()
hier_mod_output_testing_np = list_hier_score_calc(data_hier_mod, data_labels_mod, list_testing_pairs, df_questions)
hier_mod_output_train_np = list_hier_score_calc(data_hier_mod, data_labels_mod, list_learning_pairs, df_questions)

data_hier_auto, data_labels_auto = use_hierarchy_auto()
hier_auto_output_testing_np = list_hier_score_calc(data_hier_auto, data_labels_auto, list_testing_pairs, df_questions)
hier_auto_output_train_np = list_hier_score_calc(data_hier_auto, data_labels_auto, list_learning_pairs, df_questions)

d_type_hier = {"auto": (hier_auto_output_train_np, hier_auto_output_testing_np), 
               "mod": (hier_mod_output_train_np, hier_mod_output_testing_np), 
               "stat": (hier_stat_output_train_np, hier_stat_output_testing_np), 
               "manual": (hier_manual_output_train_np, hier_manual_output_testing_np)

}

# np.max(tags_output_train_np)
# np.amax(hier_output_train_np)
# np.amin(hier_output_train_np)

In [9]:
#Do the grid-search evaluation
import time
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier

random_state = 42

models = {
    # "DecisionTreeClassifier": DecisionTreeClassifier(max_depth=10, min_samples_leaf=2, random_state=random_state),
    # "SVM": SVC(kernel = 'poly', degree = 3, random_state = 42, probability = True), 
    # "KNeighborsClassifier": KNeighborsClassifier(n_neighbors=5,algorithm = 'kd_tree', weights="distance")
    "LogisticRegression": LogisticRegression(C = 0.5, penalty = 'l2', tol = 1e-8, solver = 'liblinear', random_state = 42)
}

In [10]:

def evaluate_model_hier_tags(name, output_train_np, output_testing_np, type_hier):
  hier_output_train_np, hier_output_testing_np = d_type_hier[type_hier]
  hier_train_np = np.column_stack((output_train_np, hier_output_train_np, tags_output_train_np))
  hier_testing_np = np.column_stack((output_testing_np, hier_output_testing_np, tags_output_testing_np))

  llr = LogisticRegression(random_state = 42).fit(hier_train_np,learning_labels_np)
  prediction_testing = llr.predict(hier_testing_np)

  f1_score_value = f1_score(prediction_testing, testing_labels_np, average="weighted")
  accuracy = accuracy_score(prediction_testing, testing_labels_np)
  recall = recall_score(prediction_testing, testing_labels_np)
  precision = precision_score(prediction_testing, testing_labels_np)
  d_hier = {  "results": "{} text + {}_hier + tags".format(name, type_hier),
              "accuracy": accuracy, "f1": f1_score_value, "recall": recall, "precision":precision, "coef":  list(llr.coef_[0]) }
  return d_hier

def evaluate_model_hier(name, output_train_np, output_testing_np, type_hier):
  hier_output_train_np, hier_output_testing_np = d_type_hier[type_hier]
  hier_train_np = np.column_stack((output_train_np, hier_output_train_np))
  hier_testing_np = np.column_stack((output_testing_np, hier_output_testing_np))

  llr = LogisticRegression(random_state = 42).fit(hier_train_np,learning_labels_np)
  prediction_testing = llr.predict(hier_testing_np)

  f1_score_value = f1_score(prediction_testing, testing_labels_np, average="weighted")
  accuracy = accuracy_score(prediction_testing, testing_labels_np)
  recall = recall_score(prediction_testing, testing_labels_np)
  precision = precision_score(prediction_testing, testing_labels_np)
  d_hier = {  "results": "{} text + {}_hier".format(name, type_hier),
              "accuracy": accuracy, "f1": f1_score_value, "recall": recall, "precision":precision, "coef":  list(llr.coef_[0]) }
  return d_hier

def evaluate_model_tags(name, output_train_np, output_testing_np):
  tags_train_np = np.column_stack((output_train_np, tags_output_train_np))
  tags_testing_np = np.column_stack((output_testing_np, tags_output_testing_np))


  llr = LogisticRegression(random_state = 42).fit(tags_train_np,learning_labels_np)
  # prediction_training = np.where(prediction_training >=0.5, 1, 0)
  prediction_testing = llr.predict(tags_testing_np)
  f1_score_value = f1_score(prediction_testing, testing_labels_np, average="weighted")
  accuracy = accuracy_score(prediction_testing, testing_labels_np)
  recall = recall_score(prediction_testing, testing_labels_np)
  precision = precision_score(prediction_testing, testing_labels_np)
  d_tags = {  "results": "{} text + tags".format(name),
              "accuracy": accuracy, "f1": f1_score_value, "recall": recall, "precision":precision, "coef":  list(llr.coef_[0]) }
  return d_tags

In [11]:
def evaluate_model(models):
  for name, model in models.items():
      start = time.time()
      model.fit(learning_data_np, learning_labels_np)
      stop = time.time()
      prediction = model.predict(testing_data_np)
      f1_score_value = f1_score(prediction,testing_labels_np, average="weighted")
      accuracy = accuracy_score(prediction,testing_labels_np)
      recall = recall_score(prediction,testing_labels_np)
      precision = precision_score(prediction,testing_labels_np)

      output_train_np = model.predict_proba(learning_data_np)
      output_testing_np = model.predict_proba(testing_data_np)
      output_testing_np = output_testing_np[:, 1]
      output_train_np = output_train_np[:, 1]

      f1_score_value = f1_score(np.where(output_testing_np >=0.5, 1, 0), testing_labels_np, average="weighted")
      accuracy = accuracy_score(np.where(output_testing_np >=0.5, 1, 0), testing_labels_np)
      recall = recall_score(np.where(output_testing_np >=0.5, 1, 0), testing_labels_np)
      precision = precision_score(np.where(output_testing_np >=0.5, 1, 0), testing_labels_np)
      d_text = {'results': "{} text".format(name),
                'accuracy': accuracy, 'f1': f1_score_value, 'recall': recall, 'precision':precision, 'Training time': "{}s".format(stop - start)}
      d_tags = evaluate_model_tags(name, output_train_np, output_testing_np)
      d_hier_mod = evaluate_model_hier(name, output_train_np, output_testing_np, 'mod')
      d_hier_mod_tags = evaluate_model_hier_tags(name, output_train_np, output_testing_np, 'mod')
      d_hier_stat = evaluate_model_hier(name, output_train_np, output_testing_np, 'stat')
      d_hier_stat_tags = evaluate_model_hier_tags(name, output_train_np, output_testing_np, 'stat')
      d_hier_manual = evaluate_model_hier(name, output_train_np, output_testing_np, 'manual')
      d_hier_manual_tags = evaluate_model_hier_tags(name, output_train_np, output_testing_np, 'manual')
      d_hier_auto = evaluate_model_hier(name, output_train_np, output_testing_np, 'auto')
      d_hier_auto_tags = evaluate_model_hier_tags(name, output_train_np, output_testing_np, 'auto')

      list_d = [d_text, d_tags, d_hier_mod, d_hier_mod_tags, d_hier_stat, d_hier_stat_tags, d_hier_manual, d_hier_manual_tags, d_hier_auto, d_hier_auto_tags]
      with open('/drive/MyDrive/rp_data/json/scores/rank3/{}_evaluation.json'.format(name), 'w') as fp:
        json.dump(list_d, fp)

In [12]:
evaluate_model(models)